# Load Packages

# Summary:
duplicated data: 
791 duplcates


very early years : does it make sense?

weird peak in 2004 mostly on partnerlid

biggest group: former member

need to explain: 
Unique values in 'membership_type':
[nan 'partner_member' 'life_member' 'former_member' 'family_pass_partner'
 'youth_member' 'family_pass_young_partner' 'staff_member'
 'partner_staff_member']

Unique values in 'membership_channel':
['unknown' 'dm_other' 'shops' 'lsc' 'alc_ww' 'online']

Unique values in 'cardcolor':
['platinum' 'blue' 'silver' 'bronze' nan]


In [ ]:
import os
import pandas as pd
from pathlib import Path
import warnings
import plotly.io as pio
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder


pio.renderers.default='plotly_mimetype+notebook'
pd.set_option('display.max_columns', None)
RANDOM_SEED = 69

warnings.filterwarnings('ignore')
if 'base_dir' not in globals():
    base_dir = Path.cwd()
main_dir = base_dir.parent.parent
print(f"Intended Main Directory: {main_dir}")
if Path.cwd() != main_dir:
    os.chdir(main_dir)
    print(f"Changed working directory to: {main_dir}")
else:
    print("Working directory is already set correctly.")
print("Current Working Directory:", Path.cwd())

# Config

In [ ]:
with open('confs/column_names_translations.json') as file:
    column_names_translation = json.load(file)

# Load parquet data

In [ ]:
list_data_sources = ['td_klantwaarde_basis', 'td_klantwaarde_reizen', 'td_klantwaarde_reizen_golf', 'td_klantwaarde_verzekeren']
for data_source in list_data_sources:
    data_dir = Path(f'data/client_data/{data_source}')
    full_df = pd.concat(
        (pd.read_parquet(parquet_file)
        for parquet_file in data_dir.glob('*.parquet')), ignore_index=True
    ).drop_duplicates()
    print(data_source)
    print(full_df.head())
    print(len(full_df))
    print('################')

1. the first table contains data, such as type of membership, age, zip code, first member date.
2. the second table contains all travel data and contains booking date, category and product.
3. the third table contains travel data for our golf department and this table contains product, start date and end date.
4. the fourth table contains our insurance data and also contains product, start date and end date. 

## Copy to TD Repo

In [ ]:
#!export AWS_PROFILE=td-org-data_engineering_staging 
#!aws s3 sync data/client_data/ s3://anwb-pyspark/anwb-member-score-data/

# Client Data Base

In [ ]:
data_source = 'td_klantwaarde_basis'
data_dir = Path(f'data/client_data/{data_source}')
full_df = pd.concat(
    (pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')) , ignore_index=True).drop_duplicates()
    
full_df = full_df.rename(columns=column_names_translation['datasources_columns'])
for column, mapping in column_names_translation['column_values'].items():
    full_df[column] = full_df[column].map(mapping)


## General Statistics

In [ ]:
print("\nDataset Information:")
full_df.info()
print("\nMissing values in each column:")
print(full_df.isnull().sum())
categorical_cols = ['membership_type', 'membership_channel', 'cardcolor']
for col in categorical_cols:
    print(f"\nUnique values in '{col}':")
    print(full_df[col].unique())



In [ ]:
full_df['join_date'] = pd.to_datetime(full_df['join_date'], errors='coerce')
full_df['year_joined'] = full_df['join_date'].dt.year
full_df['month_joined'] = full_df['join_date'].dt.month

In [ ]:
sorted_membership_types = full_df['membership_type'].value_counts().index.tolist()
fig1 = px.histogram(full_df, x='membership_type', color='membership_type', 
                    title='Distribution of Membership Types',
                    color_discrete_sequence=px.colors.qualitative.Vivid,
                    category_orders={'membership_type': sorted_membership_types})
fig1.update_layout(xaxis_title='Membership Type', yaxis_title='Count')
fig1.show()

sorted_card_colors = full_df['cardcolor'].value_counts().index.tolist()
fig2 = px.histogram(full_df, x='cardcolor', color='cardcolor', 
                    title='Distribution of Card Colors',
                    color_discrete_sequence=px.colors.qualitative.Pastel1,
                    category_orders={'cardcolor': sorted_card_colors})
fig2.update_layout(xaxis_title='Card Color', yaxis_title='Count')
fig2.show()

fig3 = px.histogram(full_df, x='age', nbins=30, title='Age Distribution of Members', marginal='box',
                    color_discrete_sequence=['teal'])
fig3.update_layout(xaxis_title='Age', yaxis_title='Number of Members')
fig3.show()

sorted_membership_channels = full_df['membership_channel'].value_counts().index.tolist()
fig4 = px.histogram(full_df, y='membership_channel', color='membership_channel', 
                    orientation='h',
                    title='Membership Channel Distribution',
                    color_discrete_sequence=px.colors.qualitative.Set3,
                    category_orders={'membership_channel': sorted_membership_channels})
fig4.update_layout(xaxis_title='Count', yaxis_title='Membership Channel')
fig4.show()

members_per_year = full_df.groupby('year_joined').size().reset_index(name='members_count')
fig5 = px.line(members_per_year, x='year_joined', y='members_count', title='Number of Members Joined Each Year',
               markers=True)
fig5.update_layout(xaxis_title='Year Joined', yaxis_title='Number of Members')
fig5.show()

full_df['year_month_joined'] = full_df['join_date'].dt.to_period('M').astype(str)
members_per_month = full_df.groupby('year_month_joined').size().reset_index(name='members_count')
fig6 = px.line(members_per_month, x='year_month_joined', y='members_count',
               title='Number of Members Joined Each Month')
fig6.update_layout(xaxis_title='Year-Month Joined', yaxis_title='Number of Members')
fig6.update_xaxes(nticks=20, tickangle=45)
fig6.show()

membership_by_year = full_df.groupby(['year_joined', 'membership_type']).size().reset_index(name='members_count')
fig7 = px.bar(membership_by_year, x='year_joined', y='members_count', color='membership_type',
              title='Membership Types Over the Years',
              color_discrete_sequence=px.colors.qualitative.Prism)
fig7.update_layout(xaxis_title='Year Joined', yaxis_title='Number of Members')
fig7.show()

fig8 = px.box(full_df, x='membership_type', y='age', color='membership_type',
              title='Age Distribution by Membership Type',
              color_discrete_sequence=px.colors.qualitative.Vivid)
fig8.update_layout(xaxis_title='Membership Type', yaxis_title='Age')
fig8.show()

fig9 = px.box(full_df, x='cardcolor', y='age', color='cardcolor',
              title='Age Distribution by Card Color',
              color_discrete_sequence=px.colors.qualitative.Pastel1)
fig9.update_layout(xaxis_title='Card Color', yaxis_title='Age')
fig9.show()

fig10 = px.box(full_df, x='membership_channel', y='age', color='membership_channel',
               title='Age Distribution by Membership Channel',
               color_discrete_sequence=px.colors.qualitative.Set2)
fig10.update_layout(xaxis_title='Membership Channel', yaxis_title='Age')
fig10.update_xaxes(tickangle=45)
fig10.show()


df_encoded = full_df.copy()
label_encoders = {}
categorical_cols = ['membership_type', 'membership_channel', 'cardcolor']
for column in categorical_cols:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_encoded[column])
    label_encoders[column] = le

numerical_cols = ['age', 'year_joined'] + categorical_cols

corr_matrix = df_encoded[numerical_cols].corr()

fig11 = px.imshow(corr_matrix, text_auto=True, title='Correlation Matrix of Encoded Variables')
fig11.show()

top_postcodes = full_df['postcode'].value_counts().head(20).reset_index()
top_postcodes.columns = ['postcode', 'members_count']
fig12 = px.bar(top_postcodes, x='postcode', y='members_count', title='Top 20 Postcodes by Number of Members',
               color='members_count', color_continuous_scale='Magma')
fig12.update_layout(xaxis_title='Postcode', yaxis_title='Number of Members')
fig12.show()